## Исследование надёжности заёмщиков

По статистике о платёжеспособности клиентов требуется узнать  влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

<i><b>Действия:</b></i> выведем первые 10 строк таблицу и общую информацию о таблице

In [8]:
import pandas as pd

df = pd.read_csv('./datasets/data.csv')
print(df.head(10)) 
print(df.info()) 

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

<i><b>Вывод:</i></b> После ознакомления с данными видны следующие проблемы:  

1) Столбец days_employed
 - есть отрицательные значения. Стаж не может быть отрицательным
 - есть очень большие положительные значения, больше чем возраст клиента
 - тип данных float. Стаж исчисляется в днях, в этой колонке ожидается целое число 
 - есть пропущенные значения  
 
2) Столбец education
 - есть одинаковые значения, отличающиеся регистром. Это может привести к дубликатам в данных
 
3) Столбец total_income
 - тип данных float с большим количеством знаков после запятой. Нам достаточно для анализа знать доход до рубля, поэтому можем хранить здесь целое число
 - есть пропущенные значения  
 
4) Из названия некоторых столбцов непонятно, что в них хранится. 
 - children. Непонятно хранится количество детей или просто их наличие или отсутствие
 - dob_days. Непонятно, что это возраст в годах, а не днях
 - debt. Непонятно хранится здесь сумма задолженности, количество задолженностей или просто их наличие
 - total_income. Непонятно за какой срок этот доход.
 - purpose. Непонятно какая цель хранится в этой колонке.   
 
Разберемся в этом шаге с  названиями столбцов и "странными" значениями (отрицательными и очень большими) в стаже. С остальным проблемами - в следующем шаге

<b><i>Действия:</b></i> поменяем названия колонок
 - children -> children_qty
 - dob_days -> age
 - debt -> has_debt 
 - total_income -> month_income
 - purpose -> loan_purpose


In [9]:
df.set_axis(['children_qty', 'days_employed', 'age', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'has_debt',
       'month_income', 'loan_purpose'], axis='columns', inplace=True)
print('Новые названия колонок:',df.columns)


Новые названия колонок: Index(['children_qty', 'days_employed', 'age', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type',
       'has_debt', 'month_income', 'loan_purpose'],
      dtype='object')


<b><i>Действия:</b></i> посмотрим максимальное, минимальное значение и количество отрицательных, положительных, нулевых значений в колонке стаж. Посмотрим какому типу занятости соответсвуют положительные и отрицательные значения стажа

In [10]:
print('Максимальное значение колонки days_employed:', df['days_employed'].max())
print('Минимальное значение колонки days_employed:', df['days_employed'].min())
print()
print('Количество отрицательных значений колонки days_employed:', df.loc[df['days_employed'] <0]['days_employed'].count())
print('Количество положительных значений колонки days_employed:', df.loc[df['days_employed'] >0]['days_employed'].count())
print('Количество нулевых значений колонки days_employed:', df.loc[df['days_employed'] == 0]['days_employed'].count())
print()
print('Тип занятости для записей, где стаж больше 0:',df.loc[(df['days_employed'] > 0)]['income_type'].unique())
print('Тип занятости для записей, где стаж больше 0:',df.loc[(df['days_employed'] < 0)]['income_type'].unique())

Максимальное значение колонки days_employed: 401755.40047533
Минимальное значение колонки days_employed: -18388.949900568383

Количество отрицательных значений колонки days_employed: 15906
Количество положительных значений колонки days_employed: 3445
Количество нулевых значений колонки days_employed: 0

Тип занятости для записей, где стаж больше 0: ['пенсионер' 'безработный']
Тип занятости для записей, где стаж больше 0: ['сотрудник' 'компаньон' 'госслужащий' 'студент' 'предприниматель'
 'в декрете']


<b><i>Вывод:</b></i> Выдвинем предположение, что данные для пенсионеров и безработных выгружались из одной системы, а для работников - из другой и положительные значения указаны в часах, а не в днях, а отрицательные - записаны не с тем знаком. Не будем заменять некорректные значения, т.к. не будем анализировать данные в дальнейшем по этому столбцу.    
Причиной таких ошибок могут быть:
 - некорректная обработка тире и запись его как знака минус при записи стажа в систему. Например, при преобразовании строки "стаж - 675 дней"
 - запись стажа как разницы двух дат в часах: даты подачи заявления на кредит и устройства на работу вместо дней   

### Вывод

- Изучили общую информацию о таблице. Обнаружили пропущенные значения в стаже и доходе; некорректные значения стажа; неверный тип данных для стажа и дохода; одинаковые значения, отличающиеся регистром в колонке образование; не совсем понятные названия столбцов.
- Поменяли названия столбцов на более соотносящиеся с информацией, которая там хранится
- Выдвинули гипотезу, что некорректные значения стажа записаны в часах для положительных значений и записаны со знаком минусом - для отрицательных. Возможные причины появления таких данных: обработка дефиса как минуса, запись стажа как разницы между датой подачи заявления на кредит и устройства на работу в часах вместо дней. Данные заменять не стали, т.к. не будем их использовать в дальнейшем анализе 

- Остальные ошибки, найденные в данных, исправим в шаге 2



### Шаг 2. Предобработка данных

### Обработка пропусков

In [11]:
print()
for column in df:        
    if column in ('children_qty','days_employed','age','has_debt','month_income'):
        print('Максимальное значение колонки',column,df[column].max())
        print('Минимальное значение колонки',column,df[column].min())
    else:
        print ('Уникальные значения колонки',column,df[column].unique())
    print()


Максимальное значение колонки children_qty 20
Минимальное значение колонки children_qty -1

Максимальное значение колонки days_employed 401755.40047533
Минимальное значение колонки days_employed -18388.949900568383

Максимальное значение колонки age 75
Минимальное значение колонки age 0

Уникальные значения колонки education ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

Уникальные значения колонки education_id [0 1 2 3 4]

Уникальные значения колонки family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

Уникальные значения колонки family_status_id [0 1 2 3 4]

Уникальные значения колонки gender ['F' 'M' 'XNA']

Уникальные значения колонки income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

Максимал

<b><i>Вывод:</b></i>  
 - В колонке дети есть записи с некорректным значением -1  
 - В колонке возраст есть записи с некорректным значением 0  
 - В колонке семейное положение есть записи с разным регистром

<b><i>Действия:</b></i> на прошлом шаге мы обнаружили одинакое количество пропущенных значений в колонках со стажем и доходом. Проверим, что значения в этих колонках пропущены для одних и тех же клиентов.

In [12]:
print('Строк, у которых days_employed null, а month_income - не null:',df.loc[(df['days_employed'].isnull()) & (df['month_income'].isnull() != True)]
['days_employed'].count())

Строк, у которых days_employed null, а month_income - не null: 0


<b><i>Вывод:</b></i> так как данные о стаже и доходах пропущены у одних и тех же клиентов, то одной из причин может быть, то что они еще не принесли справку с работы, где указан стаж и доход


<b><i> Действия: </b></i>так как в дальнейшем анализе мы не будем использовать данные о стаже, то заменим их на нули. Проверим, что после замены нет записей с null в стоблце стаж

In [13]:
df['days_employed'] = df['days_employed'].fillna(value=0)
print(df['days_employed'].isnull().sum())

0


<b><i>Действия:</b></i> заменим пропущенные значения доходов на медианные, расчитанные, по каждому из типов занятости. Проверим, что после замены не осталось null значений в колонке доход

In [14]:

def median_month_income(data):
    #print(data.name)
    #print(data.index)
    #print(type(data))
    median =  df.loc[(df['month_income'].isnull() == False) & (df['income_type'] == data.name) ,'month_income'].median()
    #print(data)
    df.loc[data.index,'month_income'] = median


df.loc[df['month_income'].isnull() == True].groupby('income_type')['month_income'].transform(median_month_income)


print(df['month_income'].isnull().sum())    

0


In [15]:
def median_month_income(in_df,income_type):
    median = in_df.loc[(df['month_income'].isnull() == False) & (in_df['income_type'] == income_type) ,'month_income'].median()
    print(income_type)
    print(median)
    print()
    in_df.loc[(in_df['month_income'].isnull()) & (in_df['income_type']== income_type),'month_income'] = median


unique_income_type_list = df.loc[df['month_income'].isnull()]['income_type'].unique()

for unique_income_type in unique_income_type_list:
    median_month_income(df,unique_income_type) 

print(df['month_income'].isnull().sum())    

0


<b><i>Действия:</b></i> проверим сколько пропусков в колонках дети, возраст 

In [16]:
print('Количество записей с возрастом 0:',df.loc[df['age'] == 0]['age'].count())
print('Количество записей с -1 ребенком:',df.loc[df['children_qty'] == -1]['children_qty'].count())


Количество записей с возрастом 0: 101
Количество записей с -1 ребенком: 47


<b><i>Вывод:</b></i> так как записей с возрастом клиенты равным нулю и записей примерно 0.5% от общих записей, с -1 ребенком - примерно 0.2% от общих записей, то удалим эти записи.
Возможные причины появления этих данных:  
 - незаполненные значения количества детей выгрузились в систему как -1
 - клиент заполнил количество детей как 1 в тексте "детей -1", дефис неверно обработался как минус
 - клиент не заполнил данные о возрасте, поэтому указан возраст 0
 

<b><i>Действия:</b></i> удалим записи, где -1 ребенок или нулевой возраст. Проверим, что записи удалились

In [17]:
df.drop(df[(df.age == 0) | (df.children_qty == -1)].index, inplace=True)

print('Количество записей с возрастом 0:',df.loc[df['age'] == 0]['age'].count())
print('Количество записей с -1 ребенком:',df.loc[df['children_qty'] == -1]['children_qty'].count())

Количество записей с возрастом 0: 0
Количество записей с -1 ребенком: 0


### Вывод

- Обнаружили пропущенные значения в стаже, выраженные null'ами. Заменили их на нули, т.к. таких данных много и мы не будем в дальнейшем обрабатывать данные по этому полю. Возможная причина появления пропущенных значений - то, что клиент не принес справку о доходах
- Обнаружили пропущенные значения в доходе, выраженные null'ами. Таких данных много и мы будем в дальнейшем анализировать данные по этому столбцу. Заменили их на медианные значения, расчитанные по каждому из типов занятости. Возможная причина появления пропущенных значений - то, что клиент не принес справку о доходах
- Обнаружили пропущенные значения в возрасте, записанные как 0. Таких данных очень мало относительного обшего количества данных. Удалили записи, где возраст равен 0. Возможная причина появления пропусков - клиент не заполнил данные о возрасте
- Обнаружили пропущенные значеняи в количестве детей, записанные как -1. Таких данных очень мало относительного обшего количества данных. Удалили записи, где количество детей -1. Возможная причины появления пропусков - незаполненные значения количества детей выгрузились в систему как -1; клиент заполнил количество детей как 1 в тексте "детей -1", дефис неверно обработался как минус



### Замена типа данных

<b><i>Действия: </b></i> заменим тип данных в колонках стаж и доход на целочисленный. Стаж исчисляется в количестве дней, поэтому это целое число, а доход - нам достаточно знать до рубля, поэтому мы тоже можем хранить его как целое число. Отбросим знаки после запятой у текущих значений, т.к. отклонение в рубль/день нам не важно, после чего приведем их к типу int

In [18]:
df['days_employed'] = round(df['days_employed'])
df['days_employed'] = df['days_employed'].astype('int')

df['month_income'] = round(df['month_income'])
df['month_income'] = df['month_income'].astype('int')

print(df.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21377 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children_qty      21377 non-null  int64 
 1   days_employed     21377 non-null  int32 
 2   age               21377 non-null  int64 
 3   education         21377 non-null  object
 4   education_id      21377 non-null  int64 
 5   family_status     21377 non-null  object
 6   family_status_id  21377 non-null  int64 
 7   gender            21377 non-null  object
 8   income_type       21377 non-null  object
 9   has_debt          21377 non-null  int64 
 10  month_income      21377 non-null  int32 
 11  loan_purpose      21377 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

 Отбросили знаки после запятой у текущих значений в колонках стаж и доход, т.к. отклонение в рубль/день нам не важно, после чего привели их к типу int

### Обработка дубликатов

<b><i>Действия:</b></i> Посмотрим количество дубликатов. Приведем значения в колонках образование и семейное положение к нижнему регистру. В колонке образование мы уже видели дубли из-за разного регистра, в семейном положении - их нет, но они могут появится, если кто-то добавит значение в нижнем регистре. Посмотрим количество дубликатов после преобразования с помощью метода duplicated. Данный метод показывает количество записей с одинаковыми значениями во всех колонках. Так как у нас нет id пользователя, то мы можем считать найденные дубли - одним и тем же пользователем. Удалим их и проверим, что после удаления дубликатов не осталось

In [19]:
print('Количество дублей до изменения регистра:',df.duplicated().sum())
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
print('Количество дублей после изменения регистра:',df.duplicated().sum())
df = df.drop_duplicates()
print('После удаления дублей:',df.duplicated().sum())

Количество дублей до изменения регистра: 54
Количество дублей после изменения регистра: 71
После удаления дублей: 0


### Вывод

Удалили дубли в данных, вызванные одинаковым записям с разным регистром в поле образование и другими причинами. Например, могло быть такое, что одни и те же клиенты были заведены в разных системах и выгружены в эту таблицу.

### Лемматизация

<b><i>Действия:</b></i> Для каждой уникальной цели кредитов выделим леммы с помощью библиотеки Pymystem3. Запишем значения целей и список выделенных лемм в них в новую  таблицу-словарь loan_purpose_dict. Выведем новую таблицу на экран

In [20]:
from pymystem3 import Mystem
m = Mystem()


unique_loan_purposes = df['loan_purpose'].unique()
loan_purpose_dict = pd.DataFrame(columns=['purpose','lemmatized_purpose'])

for purpose in unique_loan_purposes:
    lemmas_item = m.lemmatize(purpose)
    loan_purpose_dict.loc[len(loan_purpose_dict)] = [purpose,lemmas_item]
    
print(loan_purpose_dict)


                                   purpose  \
0                            покупка жилья   
1                  приобретение автомобиля   
2               дополнительное образование   
3                          сыграть свадьбу   
4                        операции с жильем   
5                              образование   
6                    на проведение свадьбы   
7                  покупка жилья для семьи   
8                     покупка недвижимости   
9        покупка коммерческой недвижимости   
10              покупка жилой недвижимости   
11  строительство собственной недвижимости   
12                            недвижимость   
13              строительство недвижимости   
14      на покупку подержанного автомобиля   
15            на покупку своего автомобиля   
16   операции с коммерческой недвижимостью   
17        строительство жилой недвижимости   
18                                   жилье   
19         операции со своей недвижимостью   
20                              ав

<b><i>Действия:</b></i> в дальнейшем нам потребуется категоризировать данные, исходя из целей кредита. Это будет удобно сделать на основе лемматизированных значений из словаря. Добавим в исходную таблицу колонку purpose_id, которая будет хранить соответствующий id из словаря loan_purpose_dict. Удалим из исходной таблицы колонку loan_purpose. Теперь в исходной таблице у нас хранится только id словаря с информацией о цели кредита loan_purpose_dict, а в словаре - вся информация о цели

In [21]:
def set_purpose_id(purpose):
    purpose_id = loan_purpose_dict.loc[loan_purpose_dict['purpose'] == purpose].index[0]
    return purpose_id

df['loan_purpose_id'] = df['loan_purpose'].apply(set_purpose_id)
df.drop(['loan_purpose'], axis='columns', inplace=True)

### Вывод

Выделили леммы для каждого уникального значения цели кредита. Создали словарь для хранения уникальных значений целей кредита и списка лемм, выделенных из них. Привязали к исходной таблице id цели кредита из словаря и удалили из нее колонку loan_purpose

### Категоризация данных

<b><i>Действия:</b></i> посмотрим медиану по столбцу доходы

In [22]:
print (df.month_income.mean())

165387.72998216466


<b><i>Действия:</b></i> Выделим три уровня доходов на основании прожиточного минимума и медианы: меньше прожиточного минимума, меньше среднего и больше среднего. Добавим в исходную таблицу новый столбец income_level, куда с помощью функции income_level запишем соответствующую категорию уровня дохода

In [23]:
def income_level(month_income):
    if month_income < 19797:
        return 'меньше прожиточного минимума'
    if (month_income >= 19797) and (month_income<165000):
        return 'меньше среднего'
    else:
        return 'больше среднего'
    
df['income_level'] = df['month_income'].apply(income_level)
print(df.head())

   children_qty  days_employed  age education  education_id     family_status  \
0             1          -8438   42    высшее             0   женат / замужем   
1             1          -4025   36   среднее             1   женат / замужем   
2             0          -5623   33   среднее             1   женат / замужем   
3             3          -4125   32   среднее             1   женат / замужем   
4             0         340266   53   среднее             1  гражданский брак   

   family_status_id gender income_type  has_debt  month_income  \
0                 0      F   сотрудник         0        253876   
1                 0      F   сотрудник         0        112080   
2                 0      M   сотрудник         0        145886   
3                 0      M   сотрудник         0        267629   
4                 1      F   пенсионер         0        158616   

   loan_purpose_id     income_level  
0                0  больше среднего  
1                1  меньше среднего  
2 

<b><i>Действия:</b></i> посмотрим какие слова используются в качестве цели кредита и как часто

In [24]:
from pymystem3 import Mystem
m = Mystem()
unique_loan_purposes = ' '.join(loan_purpose_dict['purpose'])
result = m.lemmatize(unique_loan_purposes)

from collections import Counter
print(Counter(result))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


<b><i>Действия:</b></i> выделим следующие категории целей кредита как самые часто встречающиеся и отражающие суть: автомобиль, свадьба, недвижимость, образование, ремонт, другое. Запишем категорию цели кредита в словарь loan_purpose_dict

In [25]:
def loan_purpose_group(purpose):
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'свадьба' in purpose:
        return 'свадьба'
    if ('недвижимость' in purpose) or ('жилье' in purpose):
        return 'недвижимость'
    if 'образование' in purpose:
        return 'образование'
    if 'ремонт' in purpose:
        return 'ремонт'
    else:
        return 'другое'

loan_purpose_dict['category_id'] = loan_purpose_dict['lemmatized_purpose'].apply(loan_purpose_group)
print (loan_purpose_dict.head())
   

                      purpose                    lemmatized_purpose  \
0               покупка жилья               [покупка,  , жилье, \n]   
1     приобретение автомобиля     [приобретение,  , автомобиль, \n]   
2  дополнительное образование  [дополнительный,  , образование, \n]   
3             сыграть свадьбу             [сыграть,  , свадьба, \n]   
4           операции с жильем        [операция,  , с,  , жилье, \n]   

    category_id  
0  недвижимость  
1    автомобиль  
2   образование  
3       свадьба  
4  недвижимость  


### Вывод

Так как нам в дальнейшем нужно будет анализировать данные по уровню дохода и цели кредита, выделили для них категории:
- меньше прожиточного минимума, меньше среднего и больше среднего для уровня доходов на основании прожиточного минимум и медины по доходам. Добавили дополнительный столбец с категорией в исходную таблицу
- автомобиль, свадьба, недвижимость, образование, ремонт, другое как самые часто встречающиеся и отражающие суть. Записали категорию цели кредита в словарь loan_purpose_dict

### Шаг 3. Ответьте на вопросы

<b><i> Действия: </b></i> перед тем как анализировать данные выделим словари для хранения значений семейного статуса и образования. В исходной таблице оставим только id для поиска по этим словарям. Это позволит уменьшить время обработки запросов

In [26]:

family_status_dict = df[['family_status_id','family_status']]
education_dict = df[['education_id','education']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
education_dict = education_dict.drop_duplicates().reset_index(drop=True)

print(family_status_dict)
print(education_dict)
df = df.drop(columns=['family_status', 'education'])
print(df.head())

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  не женат / не замужем
   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень
   children_qty  days_employed  age  education_id  family_status_id gender  \
0             1          -8438   42             0                 0      F   
1             1          -4025   36             1                 0      F   
2             0          -5623   33             1                 0      M   
3             3          -4125   32             1                 0      M   
4             0         340266   53             1                 1      F   

  income_type  has_debt  month_income  loan_purpose

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

В решении данного вопроса из тех данных, которые у нас есть, мы можем ориентироваться только на есть у клиента уже кредиты или нет. Посчитаем соотношение количества должников относительно общего количества клиентов с детьми и без детей

In [27]:
correlation_children_debt = df.groupby('children_qty',as_index=False).agg({'has_debt':['sum','count']})

debt_ratio_without_kids = correlation_children_debt.loc[correlation_children_debt['children_qty'] == 0]['has_debt']['sum'].sum() / correlation_children_debt.loc[correlation_children_debt['children_qty'] == 0]['has_debt']['count'].sum()
debt_ratio_with_kids = correlation_children_debt.loc[correlation_children_debt['children_qty'] != 0]['has_debt']['sum'].sum() / correlation_children_debt.loc[correlation_children_debt['children_qty'] == 0]['has_debt']['count'].sum()

print('Соотношение должников относительно общего количества клиентов без детей {:.2%}'.format(debt_ratio_without_kids))
print('Соотношение должников относительно общего количества клиентов с детьми {:.2%}'.format(debt_ratio_with_kids))

Соотношение должников относительно общего количества клиентов без детей 7.55%
Соотношение должников относительно общего количества клиентов с детьми 4.81%


### Вывод

У людей без детей примерно в 1.5 раза больше кредитов на одного человека. Можем сделать вывод, что зависимость между наличием детей и возвратом кредита в срок есть и люди с детьми с большей вероятностью вернут кредит в срок

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Посчитаем соотношение количества должников относительно общего количества клиентов для каждого семейного положения

In [28]:
correlation_family_status_debt =  df.groupby('family_status_id', as_index=False).agg({'has_debt':['sum','count']})
correlation_family_status_debt['ratio'] = (correlation_family_status_debt['has_debt']['sum']/correlation_family_status_debt['has_debt']['count']) 
correlation_family_status_debt.columns = ['family_status_id', 'sum','count','ratio']
correlation_family_status_debt_full = pd.merge(correlation_family_status_debt, family_status_dict,on='family_status_id', how='inner')

def print_result(row):
    print('Соотношение должников относительно общего количества клиентов для группы {} {:.2%}'.format(row['family_status'],row['ratio']))
    
result = correlation_family_status_debt_full.sort_values(by='ratio').apply(print_result,axis=1)

Соотношение должников относительно общего количества клиентов для группы вдовец / вдова 6.53%
Соотношение должников относительно общего количества клиентов для группы в разводе 7.20%
Соотношение должников относительно общего количества клиентов для группы женат / замужем 7.55%
Соотношение должников относительно общего количества клиентов для группы гражданский брак 9.36%
Соотношение должников относительно общего количества клиентов для группы не женат / не замужем 9.79%


### Вывод

Из полученных данных можем сделать вывод, что зависимость между семейным положением и возвратом кредита в срок есть. Вероятнее всего вернет кредит вдовец/вдова и с наименьшей вероятностью человек, который не женат/не замужем или состоит в гражданском браке. Примерно с одинаковой вероятностью вернут кредит люди в разводе или те, кто женат/замужем.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Посчитаем соотношение количества должников относительно общего количества клиентов для каждого уровня дохода

In [29]:
correlation_income_level_debt = df.groupby('income_level', as_index = False).agg({'has_debt':['sum','count']})
correlation_income_level_debt['ratio'] = (correlation_income_level_debt['has_debt']['sum']/correlation_income_level_debt['has_debt']['count'])
correlation_income_level_debt.columns = ['income_level', 'sum','count','ratio']

def print_result(row):
    print('Соотношение должников относительно общего количества клиентов для группы с доходом {} {:.2%}'.format(row['income_level'],row['ratio']))

result = correlation_income_level_debt.sort_values(by='ratio').apply(print_result,axis=1)

Соотношение должников относительно общего количества клиентов для группы с доходом больше среднего 7.69%
Соотношение должников относительно общего количества клиентов для группы с доходом меньше среднего 8.40%


### Вывод

Соотношение количества должников относительно общего количества клиентов для каждого уровня дохода отличается несильно, поэтому можем сделать вывод, что доход не влияеи на возврат кредита в срок

- Как разные цели кредита влияют на его возврат в срок?

Посчитаем соотношение количества должников относительно общего количества клиентов для каждой из целей кредита

In [30]:
loan_purpose_dict.index.name = 'loan_purpose_id'

loan_purpose_full = pd.merge(loan_purpose_dict, df,on='loan_purpose_id', how='inner')
correlation_loan_purpose_debt = loan_purpose_full.groupby('category_id', as_index = False).agg({'has_debt':['sum','count']})
correlation_loan_purpose_debt['ratio'] = (correlation_loan_purpose_debt['has_debt']['sum']/correlation_loan_purpose_debt['has_debt']['count'])
correlation_loan_purpose_debt.columns = ['category_id', 'sum','count','ratio']

def print_result(row):
    print('Соотношение должников относительно общего количества клиентов для группы {} {:.2%}'.format(row['category_id'],row['ratio']))
result = correlation_loan_purpose_debt.sort_values(by='ratio').apply(print_result,axis=1)

Соотношение должников относительно общего количества клиентов для группы недвижимость 7.25%
Соотношение должников относительно общего количества клиентов для группы свадьба 7.97%
Соотношение должников относительно общего количества клиентов для группы образование 9.28%
Соотношение должников относительно общего количества клиентов для группы автомобиль 9.34%


### Вывод

С наибольшей вероятностью вернут кредит люди, которые берут его на недвижимость или свадьбу и с меньшей вероятностью те, кто берет его на образование или автомобиль

### Шаг 4. Общий вывод

В результате все гипотезы кроме одной подтверждены. 
- Зависимость между наличием детей и возвратом кредита в срок. Люди с детьми с большей вероятностью вернут кредит в срок
- Зависимость между семейным положением и возвратом кредита в срок есть 
- Зависимость между уровнем дохода и возвратом кредита в срок нет
- С наибольшей вероятностью вернут кредит люди, которые берут его на недвижимость или свадьбу и с меньшей вероятностью те, кто берет его на образование или автомобиль

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.